<a href="https://colab.research.google.com/github/ssonti1809/Dim_Red/blob/main/Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 5 (Winter 2025)
## PageRank

In [29]:
from IPython.display import Image
print("Colab 5 Mascot")
Image(url='https://media.giphy.com/media/cCOVfFwDI3awdse5A3/giphy.gif',width=150)

Colab 5 Mascot


### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [30]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [31]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks ([tutorial](https://networkx.org/documentation/stable/tutorial.html)).

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [ ]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

#### Function to print graph information

In [ ]:
def print_graph_info(G, directed=True):
  print("Number of nodes:", len(G.nodes))
  print("Number of edges:", len(G.edges))
  if directed:
    print("Average in-degree:", sum(dict(G.in_degree).values()) / len(G.nodes))
    print("Average out-degree:", sum(dict(G.out_degree).values()) / len(G.nodes))
  else:
    print("Average degree:", sum(dict(G.degree).values()) / len(G.nodes))

In [ ]:
print_graph_info(G, True)

### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  **From now on, use this connected component for all the following tasks.** [[More Info]](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.weakly_connected_components.html)

Print its information.

In [ ]:
''' 3 lines of code in total expected. Execution time should be less than 30 seconds'''

# YOUR CODE HERE
largest_weak_cc = max(nx.weakly_connected_components(G), key=len)
Gcc = G.subgraph(largest_weak_cc).copy()
print_graph_info(Gcc, True)

Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [ ]:
''' 1 line of code in total expected. Execution time should be around or less than 15 seconds.'''

# YOUR CODE HERE
pagerank_scores = nx.pagerank(Gcc)


In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using the parameters ```seed = 1``` and ```directed=False``` where applicable, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess (the value that is closest to and satisfies $\geq$)** the number of edges in the original connected component (the largest weakly connected component you attained above)

and compute the PageRank vectors for both graphs. Print generated graph's information, if needed.


In [ ]:
''' 6-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.
The execution time for one pagerank computation should be around 20 seconds.'''

# YOUR CODE HERE
import networkx as nx
from networkx.generators import fast_gnp_random_graph, barabasi_albert_graph
import random
random.seed(1)
sampled_nodes = random.sample(list(G.nodes), 20000)
G_sampled = G.subgraph(sampled_nodes).copy()
n = len(G_sampled.nodes)
m = int(G_sampled.number_of_edges() / n) + 1
G_random = fast_gnp_random_graph(n, 0.0002, seed=1)
G_ba = barabasi_albert_graph(n, m, seed=1)
pagerank_random = nx.pagerank(G_random, max_iter=5, tol=1e-2)
pagerank_ba = nx.pagerank(G_ba, max_iter=5, tol=1e-2)
pagerank_G = nx.pagerank(G_sampled, max_iter=5, tol=1e-2)

print("Original Sampled Graph:", n, "nodes,", G_sampled.number_of_edges(), "edges")
print("Random Graph:", G_random.number_of_nodes(), "nodes,", G_random.number_of_edges(), "edges")
print("BA Graph:", G_ba.number_of_nodes(), "nodes,", G_ba.number_of_edges(), "edges")


Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component (the largest weakly connected component you attained above).
**Sort** the components of each vector by value, and use cosine similarity as similarity measure.

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [ ]:
''' 8-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''

# YOUR CODE HERE
import numpy as np
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
pagerank_G = pagerank(G_sampled, max_iter=5, tol=1e-2)
def vector_from_pagerank_dict(pr_dict):
    return np.array([pr_dict[k] for k in sorted(pr_dict.keys())])
v_orig = vector_from_pagerank_dict(pagerank_G)
v_rand = vector_from_pagerank_dict(pagerank_random)
v_ba   = vector_from_pagerank_dict(pagerank_ba)
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
print("Cosine Similarity (Original vs Random):", cosine_similarity(v_orig, v_rand))
print("Cosine Similarity (Original vs BA):", cosine_similarity(v_orig, v_ba))



Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!